In [60]:
from okx import PublicData

public = PublicData.PublicAPI(flag='0')
data = public.get_instruments('SWAP')['data']
names = [x['instId'] for x in data if  "USDT-SWAP" in x['instId']]

In [61]:
# from okx import TradingData

# api = TradingData.TradingDataAPI(flag='0')
# api.get_margin_lending_ratio(ccy="ETH")

In [62]:
import okx.MarketData as MarketData
import pandas as pd

market = MarketData.MarketAPI(flag='0')
def get_candles(instId='ETH-USDT-SWAP', bar='1H', limit="100"):
    result = market.get_candlesticks(instId=instId, bar=bar, limit=limit)

    data = result['data']  # 列表，每项：[ts, o, h, l, c, vol, volCcy, ...]
    df = pd.DataFrame(data, columns=[
        'ts', 'open', 'high', 'low', 'close', 
        'vol', 'volCcy', 'volCcyQuote', 'confirm'
    ])
    df = df.loc[:, ["open", "high", "low", "close"]]
    df = df.astype({'open':float, 'high':float, 'low':float, 'close':float})
    df = df[::-1].reset_index(drop=True)    # 最后一行为新k

    return df

In [63]:
import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

In [64]:
candles_1H = get_candles()
[open, high, low, close] = candles_1H.iloc[-1]
abs(open - close) / open * 100

0.6894039646576076

In [ ]:
import time

def main():
    for name in names:
        time.sleep(0.5)
        candles_1H = get_candles(name, bar='1H')
        supertrend_1H = supertrend(candles_1H)
        supertrend_1H = supertrend_1H["in_uptrend"]
        current_trend_1H = supertrend_1H.iloc[-1]
        prev_trend_1H = supertrend_1H.iloc[-2]


        # 上升趋势转为下跌趋势
        # if current_trend_1H == False and prev_trend_1H == True:
        #     print(name, flush=True)

        # 涨跌幅大于20%
        [open, high, low, close] = candles_1H.iloc[-1, :4]
        rate = abs(open - close) / open * 100
        print(name, rate, flush=True)
        
        if rate > 20:
            print(name, rate, flush=True)


In [66]:
while True:
    main()
    print("main完成")

BTC-USDT-SWAP 0.6417562140453773
ETH-USDT-SWAP 0.689069789202119
OKB-USDT-SWAP 0.13799448022078334
SOL-USDT-SWAP 0.5248926355972615
DOGE-USDT-SWAP 1.2609827530100843
XRP-USDT-SWAP 0.45556865687640435
BCH-USDT-SWAP 0.4683057367452675
PUMP-USDT-SWAP 0.05246589716684852
WLFI-USDT-SWAP 0.4861111111110962
LINEA-USDT-SWAP 0.5656819610307916
0G-USDT-SWAP 0.25409086289256494
1INCH-USDT-SWAP 0.6954102920723232
2Z-USDT-SWAP 0.08340283569641606
A-USDT-SWAP 0.43183220234423575
AAVE-USDT-SWAP 0.9942895532415114
ACE-USDT-SWAP 0.14641288433382557
ACH-USDT-SWAP 0.40634421287194444
ACT-USDT-SWAP 0.9090909090909229
ADA-USDT-SWAP 0.8333333333333214
AERO-USDT-SWAP 0.7897111913357409
AEVO-USDT-SWAP 0.3419253024723782
AGLD-USDT-SWAP 0.8393742846241815
AIXBT-USDT-SWAP 0.723981900452501
ALGO-USDT-SWAP 0.5333333333333362
ALLO-USDT-SWAP 0.18099547511312736
ANIME-USDT-SWAP 0.0
APE-USDT-SWAP 0.5010020040080164
API3-USDT-SWAP 0.18369690011481582
APR-USDT-SWAP 0.47238964000651346
APT-USDT-SWAP 0.05892751915143723
A

KeyboardInterrupt: 